In [4]:

from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments

# base_model = "meta-llama/Llama-3.2-3B-Instruct"
# base_model = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"

# base_model = "bartowski/DeepSeek-R1-Distill-Qwen-14B-GGUF"
# base_model = "enferAI/DeepSeek-R1-Distill-Qwen-14B-FP8"

base_model = "unsloth/DeepSeek-R1-Distill-Qwen-14B-bnb-4bit"
# model = AutoModelForCausalLM.from_pretrained(base_model)
# print(model.get_memory_footprint())

# dataset = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k", split="train")

import json
from datasets import Dataset

with open("tarot_qa_finetuning_dataset_cleaned.json", "r") as f:
    dataset = json.load(f)

dataset = Dataset.from_list(dataset)
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(len(dataset['question']))



tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

3900


Model requirement

"bos_token": "<｜begin▁of▁sentence｜>",
  "chat_template": "{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% set ns = namespace(is_first=false, is_tool=false, is_output_first=true, system_prompt='') %}{%- for message in messages %}{%- if message['role'] == 'system' %}{% set ns.system_prompt = message['content'] %}{%- endif %}{%- endfor %}{{bos_token}}{{ns.system_prompt}}{%- for message in messages %}{%- if message['role'] == 'user' %}{%- set ns.is_tool = false -%}{{'<｜User｜>' + message['content']}}{%- endif %}{%- if message['role'] == 'assistant' and message['content'] is none %}{%- set ns.is_tool = false -%}{%- for tool in message['tool_calls']%}{%- if not ns.is_first %}{{'<｜Assistant｜><｜tool▁calls▁begin｜><｜tool▁call▁begin｜>' + tool['type'] + '<｜tool▁sep｜>' + tool['function']['name'] + '\\n' + '```json' + '\\n' + tool['function']['arguments'] + '\\n' + '```' + '<｜tool▁call▁end｜>'}}{%- set ns.is_first = true -%}{%- else %}{{'\\n' + '<｜tool▁call▁begin｜>' + tool['type'] + '<｜tool▁sep｜>' + tool['function']['name'] + '\\n' + '```json' + '\\n' + tool['function']['arguments'] + '\\n' + '```' + '<｜tool▁call▁end｜>'}}{{'<｜tool▁calls▁end｜><｜end▁of▁sentence｜>'}}{%- endif %}{%- endfor %}{%- endif %}{%- if message['role'] == 'assistant' and message['content'] is not none %}{%- if ns.is_tool %}{{'<｜tool▁outputs▁end｜>' + message['content'] + '<｜end▁of▁sentence｜>'}}{%- set ns.is_tool = false -%}{%- else %}{% set content = message['content'] %}{% if '</think>' in content %}{% set content = content.split('</think>')[-1] %}{% endif %}{{'<｜Assistant｜>' + content + '<｜end▁of▁sentence｜>'}}{%- endif %}{%- endif %}{%- if message['role'] == 'tool' %}{%- set ns.is_tool = true -%}{%- if ns.is_output_first %}{{'<｜tool▁outputs▁begin｜><｜tool▁output▁begin｜>' + message['content'] + '<｜tool▁output▁end｜>'}}{%- set ns.is_output_first = false %}{%- else %}{{'\\n<｜tool▁output▁begin｜>' + message['content'] + '<｜tool▁output▁end｜>'}}{%- endif %}{%- endif %}{%- endfor -%}{% if ns.is_tool %}{{'<｜tool▁outputs▁end｜>'}}{% endif %}{% if add_generation_prompt and not ns.is_tool %}{{'<｜Assistant｜><think>\\n'}}{% endif %}",
  "clean_up_tokenization_spaces": false,
  "eos_token": "<｜end▁of▁sentence｜>",
  "extra_special_tokens": {},
  "legacy": true,
  "model_max_length": 131072,
  "pad_token": "<|vision_pad|>",
  "padding_side": "left",
  "sp_model_kwargs": {},
  "tokenizer_class": "LlamaTokenizerFast",
  "unk_token": null,
  "use_default_system_prompt": false
}

In [ ]:

def formatting_prompts_func(example):
    output_texts = []
    # for i in range(len(example['question'])):
    #     text = f"### Question: {example['question'][i]}\n ### Answer: {example['answer'][i]}"
    #     output_texts.append(text)
    # return output_texts
    system_prompt = "You are a tarot reading assistant, your goal is to generate the best response to user's choosen card in 4 sentences"
    for i in range(len(example['question'])):
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": example['question'][i]},
            {"role": "assistant", "content": example['answer'][i]}
        ]
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        output_texts.append(text)
    return output_texts

# response_template = " ### Answer:"
response_template = "<｜Assistant｜>"  # specifically for qwen
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)

model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config)
print(model.get_memory_footprint())

peft_config = LoraConfig(r=32,
                        lora_alpha=64,
                        lora_dropout=0.05,
                        bias="none",
                        task_type="CAUSAL_LM"
                      )



import sys
import argparse

def main():
    # parser = argparse.ArgumentParser(description="Training arguments", allow_abbrev=False)


    # parser.add_argument("--epochs", type=int, default=5, help="Number of epochs")
    # parser.add_argument("--batch_size", type=int, default=help="Batch size")

    # # Parse arguments
    # args = parser.parse_args()

    epoch = 30 if args.epochs is None else args.epochs
    batch_size = 2 if args.batch_size is None else args.batch_size

    args = TrainingArguments(
        output_dir='qwen_sft_4b_v2',
        warmup_steps=1,
        num_train_epochs=5, # adjust based on the data size
        per_device_train_batch_size=4, # use 4 if you have more GPU RAM
        gradient_accumulation_steps=4,
        save_strategy="epoch", #steps
        logging_steps=100,
        optim="paged_adamw_32bit",
        learning_rate=2.5e-5,
        fp16=True,
        seed=42,
        # save_steps=50,  # Save checkpoints every 50 steps
        do_eval=False,   
        )

    trainer = SFTTrainer(
        model,
        tokenizer=tokenizer,
        train_dataset=dataset,
        args=args,
        formatting_func=formatting_prompts_func,
        data_collator=collator,
        peft_config=peft_config
    )


    trainer.train()


if __name__ == "__main__":
    main()

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
